# DeMo Preprocessed Data Visualization

This notebook provides interactive visualization capabilities for the preprocessed DeMo autonomous vehicle trajectory data. You can explore different scenarios, visualize agent trajectories, map elements, and analyze the data interactively.

## 1. Import Required Libraries

Import necessary libraries and the custom visualization module.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
import sys
import warnings
from typing import List, Optional
import importlib
warnings.filterwarnings('ignore')

# Add the project root to Python path
project_root = Path.cwd().resolve()
if not (project_root / "src").exists():
    for parent in project_root.parents:
        if (parent / "src").exists():
            project_root = parent
            break

if (project_root / "src").exists() and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import and reload the module
import src.utils.data_visualization as viz_module
importlib.reload(viz_module)

# Import custom visualization module
from src.utils.data_visualization import (
    DataSelector, DataLoader, TrajectoryVisualizer, EgoOtherVisualizer,
    DEFAULT_VIZ_CONFIG, display_agent_information, display_scenario_statistics,
    get_available_files, display_file_info, explore_scenario,
    plot_ego_velocity_analysis, create_velocity_plots, create_integrated_ego_analysis,
    )

# Try to import ipywidgets for interactive controls
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    print("Warning: ipywidgets not available. Interactive controls will be limited.")
    WIDGETS_AVAILABLE = False

# Set matplotlib style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ All required libraries imported successfully!")
print("📦 Visualization utilities loaded from data_visualization.py")

## 2. Set Up Data Paths and Parameters

Define the base path to DeMo_processed directory and check available data.

In [ ]:
# Base path to preprocessed data
BASE_DATA_PATH = Path("data/DeMo_processed")

# Check if data directory exists
if not BASE_DATA_PATH.exists():
    print(f"ERROR: Data directory {BASE_DATA_PATH} not found!")
    print("Please ensure the preprocessed data is available in the correct location.")
else:
    print(f"Data directory found: {BASE_DATA_PATH}")
    
    # Check available splits
    available_splits = []
    for split in ['train', 'val', 'test']:
        split_path = BASE_DATA_PATH / split
        if split_path.exists():
            available_splits.append(split)
            
    print(f"Available data splits: {available_splits}")

## 3. Load Available Data Files

Scan the DeMo_processed directory to discover all available preprocessed data files.

In [ ]:
# Load files for each available split
all_files = {}
for split in available_splits:
    files = get_available_files(BASE_DATA_PATH, split)
    all_files[split] = files
    print(f"\n{split.upper()} split:")
    display_file_info(files)

## 4. Initialize Visualization Components

Create instances of all visualization components.

In [ ]:
# Create visualization components
data_selector = DataSelector(all_files)
data_loader = DataLoader()
visualizer = TrajectoryVisualizer(data_loader, DEFAULT_VIZ_CONFIG)
ego_other_visualizer = EgoOtherVisualizer(data_loader, DEFAULT_VIZ_CONFIG)

print("=== Visualization Components Initialized ===")
print(f"Available splits: {list(all_files.keys())}")
for split in all_files.keys():
    print(f"{split}: {len(all_files[split])} files")

## 5. File Selection Methods

The DataSelector provides multiple ways to select files for visualization.

In [ ]:
# === FILE SELECTION METHODS ===
# The DataSelector provides multiple ways to select files:

print("=== Available File Selection Methods ===\n")

# Method 1: Random selection
print("1️⃣ Random Selection:")
print("   file_path = data_selector.select_random_file('train')")
print()

# Method 2: By index
print("2️⃣ Selection by Index:")
print("   file_path = data_selector.select_file_by_index('train', 0)  # First file")
print("   file_path = data_selector.select_file_by_index('train', 5)  # Sixth file")
print()

# Method 3: By filename
print("3️⃣ Selection by Filename:")
print("   file_path = data_selector.select_file_by_name('0a0d1522-549f-42f3-8e31-37eb8680d36e.pt', 'train')")
print("   file_path = data_selector.select_file_by_name('0a0d1522-549f-42f3-8e31-37eb8680d36e', 'train')  # .pt optional")
print()

# Method 4: Sequential navigation
print("4️⃣ Sequential Navigation:")
print("   file_path = data_selector.select_next_file('train')  # Next file in sequence")
print("   file_path = data_selector.select_previous_file('train')  # Previous file")
print()

# Method 5: List files
print("5️⃣ List Available Files:")
print("   data_selector.list_files('train', max_files=20)  # Show first 20 files")
print()

# Method 6: Get file index
print("6️⃣ Get File Index by Name:")
print("   index = data_selector.get_file_index('filename.pt', 'train')")
print()

# Example: List first 10 files
print("\n" + "="*60)
print("Example: Listing first 10 training files:")
print("="*60)
data_selector.list_files('train', max_files=10)

## 5.5. List Files with Enhanced Display

Use the `list_files_with_status` function for better file listing.

In [ ]:
# === ENHANCED FILE LISTING ===
# Note: list_files_with_status is now in scenario_classifier.py module
# For classification features, see scenario_classifier.ipynb

print("=== File Listing ===\n")

# Get train files
if 'train' in all_files:
    train_files = all_files['train']
    
    # Simple file listing
    print(f"📁 Available Training Files (Total: {len(train_files)}):")
    print(f"Showing first 15 files:\n")
    
    for i, file_path in enumerate(train_files[:15]):
        print(f"  [{i:3d}] {file_path.name}")
    
    if len(train_files) > 15:
        print(f"\n  ... and {len(train_files) - 15} more files")
    
    print("\n💡 For classification features with status display,")
    print("   see scenario_classifier.ipynb")
else:
    print("No train files available")

## 6. Quick Visualization Examples

Use the methods below to quickly visualize different scenarios.

In [ ]:
# === QUICK VISUALIZATION EXAMPLE ===
# Modify the parameters below to visualize different scenarios

# Choose your selection method:
SELECTION_METHOD = 'index'  # Options: 'random', 'index', 'filename', 'next'

# Parameters for each method:
FILE_INDEX = 0  # For 'index' method
FILE_NAME = '0000b0f9-99f9-4a1f-a231-5be9e4c523f7'  # For 'filename' method (e.g., '0a0d1522-549f-42f3-8e31-37eb8680d36e')
SPLIT = 'train'  # Data split to use

# Visualization options:
SHOW_LANES = True
SHOW_AGENT_IDS = True
SHOW_VELOCITY = True
VELOCITY_MODE = 'integrated'  # Options: 'integrated', 'separate'

print(f"=== Quick Visualization - {SELECTION_METHOD.upper()} Method ===\n")

# Select file based on method
if SELECTION_METHOD == 'random':
    file_path = data_selector.select_random_file(SPLIT)
elif SELECTION_METHOD == 'index':
    file_path = data_selector.select_file_by_index(SPLIT, FILE_INDEX)
elif SELECTION_METHOD == 'filename':
    if not FILE_NAME:
        print("❌ Error: FILE_NAME is empty. Please set a filename.")
        file_path = None
    else:
        file_path = data_selector.select_file_by_name(FILE_NAME, SPLIT)
elif SELECTION_METHOD == 'next':
    file_path = data_selector.select_next_file(SPLIT)
else:
    print(f"❌ Error: Unknown selection method '{SELECTION_METHOD}'")
    file_path = None

# Visualize if file was selected
if file_path:
    # Load data
    data_loader.load_scenario(file_path)
    
    # Print summary
    print(f"\n📊 Visualizing: {file_path.name}")
    data_loader.print_summary()
    print()
    
    # Display agent information
    # display_agent_information(data_loader)
    
    # Create visualization
    print(f"🎨 Creating visualization...")
    
    # Use the velocity mode to determine visualization type
    if SHOW_VELOCITY and VELOCITY_MODE == 'integrated':
        # Use integrated ego analysis (includes velocity)
        create_integrated_ego_analysis(
            data_loader, ego_other_visualizer,
            show_lanes=SHOW_LANES,
            show_agent_ids=SHOW_AGENT_IDS,
            show_velocity=True
        )
    elif SHOW_VELOCITY and VELOCITY_MODE == 'separate':
        # Show ego vs others first
        ego_other_visualizer.plot_ego_vs_others(
            show_lanes=SHOW_LANES,
            show_agent_ids=SHOW_AGENT_IDS
        )
        # Then show velocity analysis separately
        print("\n=== Velocity Analysis ===")
        velocity_data = plot_ego_velocity_analysis(data_loader, show_acceleration=True)
        if velocity_data is not None:
            create_velocity_plots(velocity_data)
    else:
        # Standard ego vs others visualization without velocity
        ego_other_visualizer.plot_ego_vs_others(
            show_lanes=SHOW_LANES,
            show_agent_ids=SHOW_AGENT_IDS
        )
else:
    print("❌ No file selected. Please check your parameters.")